In [1]:
from ase.io import read
vasp=read('optimized.vasp',format='vasp')
natom=len(vasp.positions)
cell=vasp.cell
positions=vasp.positions
masses=vasp.get_masses()
nmodes=natom*3
import numpy as np
lammps_dyn=np.loadtxt('Dyn.form').reshape((nmodes,nmodes))

In [2]:
eigenvalue, eigenvector=np.linalg.eig(lammps_dyn)

LAMMPSのregularオプションで出てくるものは、
$D=\frac{\Psi^{\alpha\beta}_{ij}}{\sqrt{MiMj}}$
で
$\Psi^{\alpha\beta}_{ij}=\frac{\partial^2 E_{tot}}{\partial u_{i\alpha} \partial u_{j\beta}}$
となっている。
unit=metalの場合、変位$u$はAngstrom, EnergyはeVで扱われていて、原子量はg/molの単位になっている。
固有値は周波数$\omega$の二乗だが、Hz=$1/s$の単位にするためには、Dが$1/s^2$の単位を持つようにあわせないといけない。
これを実現するために、
$\frac{mol \cdot eV}{g \cdot Angstrom\cdot Angstrom} \rightarrow kg_{to}g (m_{to}Ang)^2 N_{A} eV_{to}J \quad 1/s^2$
のような変換を行う。$N_A$はアボガドロ数である。
$kg_{to}g (m_{to}Ang)^2$で1.0e23が出てくる。THzで換算するには1THz=1e-12Hzを考えると、Dには
$0.1\cdot N_{A} eV_{to}J$
のスケール因子をかければいい

In [18]:
#scale to convert THz unit
#Boltzmanns constant (J/K)
boltz = 1.38066244e-23

#Planck's constant (Js)
planck = 6.62617636e-34

#Speed of light (in cm/s)
speedl = 2.99792458e10

#eV -> Joules

evtoj = 1.6021917e-19

#Avogadros number
avogadro = 6.022045e23

#THz:1e-12
#AngAngkgTHzTHz
fscale =0.1*evtoj*avogadro
eigenvalue*fscale

array([16020.47996374, 15318.35691059, 14903.39722164, ...,
        1004.45472347,  1012.81310909,   944.51780334])

Dynamical Matrixの対角化で出てくる固有値をsqrtしたものは、各周波数なので、THzで表示するためには$2\pi$で割る

In [29]:
omega=[]
for eig in eigenvalue*fscale:
    if eig<0.0:
        omega.append(np.sqrt(-eig)/2.0/np.pi)
    else:
        omega.append(np.sqrt(eig)/2.0/np.pi)

In [30]:
omega=np.array(omega)

In [31]:
THzTocm=1.0e12/speedl
omega_cm=omega*THzTocm

In [32]:
np.max(omega_cm)

671.9503589701478

In [33]:
sorted=np.sort(omega_cm)

In [34]:
sorted

array([1.19094034e-02, 1.24880509e-02, 2.04923770e-02, ...,
       6.48100068e+02, 6.57060744e+02, 6.71950359e+02])

In [35]:
freq_gulp=np.loadtxt('freq.dat')

In [36]:
np.max(np.abs(sorted-freq_gulp))

0.07379355841578672